In [1]:
import math
import pandas as pd
import numpy as np
import feather
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn import metrics
from sklearn import ensemble
from sklearn import grid_search

Importing trip data for March 2016 and Weather data from Weather Underground. 

The source data is read in a separate Python class. It is manipulated as a DataFrame and finally exported into Feather format. The exported file is imported in this notebook. To read more about Feather, read https://blog.rstudio.org/2016/03/29/feather/

The benefit of doing this were:
- The ETL part of the analysis is in a separate file (CitiBike_ETL.py)
- The Feather file can read in R also
- During analysis, this notebook will be run multiple times and it is efficient to just import the DataFrame

In [3]:
bikedata = feather.read_dataframe('../../Data/CitiBike_Data/bikedata.feather')
print bikedata.columns
print len(bikedata)

Index([u'bikeid', u'birth year', u'dtstartdatehour', u'end station id',
       u'end station latitude', u'end station longitude', u'end station name',
       u'female', u'male', u'start station id', u'start station latitude',
       u'start station longitude', u'start station name', u'starttime',
       u'stoptime', u'tripduration', u'usertype'],
      dtype='object')
17229335


In [4]:
bikedata.groupby(by='start station name').bikeid.count().sort_values(ascending=False).head(10)

start station name
8 Ave & W 31 St             194913
Lafayette St & E 8 St       180439
W 21 St & 6 Ave             164471
E 17 St & Broadway          159590
Pershing Square North       148941
Broadway & E 14 St          135572
Broadway & E 22 St          127350
8 Ave & W 33 St             123727
Greenwich Ave & 8 Ave       123036
Cleveland Pl & Spring St    120679
Name: bikeid, dtype: int64

In [6]:
# How many bikes will leave a particular station at a particular time?
# First let's start with a single station. In our case, let's start with E 40 St & 5 Ave
# dfFocusStation = bikedata[(bikedata['start station name'] == "8 Ave & W 31 St") 
#                           & (bikedata['usertype'] == 'Subscriber')]
dfFocusStation = bikedata[(bikedata['start station name'] == "8 Ave & W 31 St")]
dfGroupBy = dfFocusStation.groupby(by=['dtstartdatehour'])

# Number of departures per hour
departures = dfGroupBy.bikeid.count().reset_index()

# startdatehour - to join with hourly weather data
departures['startdatehour'] = departures.dtstartdatehour.apply(lambda x:x.strftime('%Y-%m-%d %H:%M:%S'))
# startdate - to join with public holiday data
departures['startdate'] = departures.dtstartdatehour.dt.date.apply(lambda x:x.strftime('%Y-%m-%d'))

# Male to female ratio
sum_male = dfGroupBy.male.sum().reset_index().male
sum_female = dfGroupBy.female.sum().reset_index().female
departures['male_to_female_ratio'] = (sum_male).astype(float) / (sum_male + sum_female)

print departures.head()

      dtstartdatehour  bikeid        startdatehour   startdate  \
0 2014-03-01 00:00:00       1  2014-03-01 00:00:00  2014-03-01   
1 2014-03-01 02:00:00       1  2014-03-01 02:00:00  2014-03-01   
2 2014-03-01 04:00:00       1  2014-03-01 04:00:00  2014-03-01   
3 2014-03-01 07:00:00       3  2014-03-01 07:00:00  2014-03-01   
4 2014-03-01 08:00:00       1  2014-03-01 08:00:00  2014-03-01   

   male_to_female_ratio  
0                     0  
1                     1  
2                     1  
3                     1  
4                     1  


In [7]:
# Import holiday data
holiday = pd.read_csv('./data/public_holiday.csv')

# Merge with departures dataframe
holiday_df = pd.merge(departures, holiday, left_on='startdate', right_on = 'Date', how='left')
holiday_df.Holiday.fillna(0, inplace=True)
holiday_df['dayofweek'] = holiday_df['dtstartdatehour'].dt.dayofweek.values

# Merging weekend and holiday produce better R2 score for linear regression but no chnage for Random Forest
holiday_df.loc[holiday_df.dayofweek <= 4, 'day_type_weekday'] = 1 #'weekday'
holiday_df.loc[holiday_df.dayofweek > 4, 'day_type_holiday'] = 1 #'weekend'
holiday_df.loc[holiday_df.Holiday <> 0, 'day_type_weekday'] = 0 #'holiday' 
holiday_df.loc[holiday_df.Holiday <> 0, 'day_type_holiday'] = 1 #'holiday' 

holiday_df.drop('Date', axis=1, inplace=True)
holiday_df.drop('Holiday', axis=1, inplace=True)
holiday_df.drop('dayofweek', axis=1, inplace=True)
holiday_df.drop('dtstartdatehour', axis=1, inplace=True)
holiday_df.drop('startdate', axis=1, inplace=True)

holiday_df.day_type_weekday.fillna(0, inplace=True)
holiday_df.day_type_holiday.fillna(0, inplace=True)
holiday_df.head(10)

,bikeid,startdatehour,male_to_female_ratio,day_type_weekday,day_type_holiday
0,1,2014-03-01 00:00:00,0.000000,0,1
1,1,2014-03-01 02:00:00,1.000000,0,1
2,1,2014-03-01 04:00:00,1.000000,0,1
3,3,2014-03-01 07:00:00,1.000000,0,1
4,1,2014-03-01 08:00:00,1.000000,0,1
5,4,2014-03-01 09:00:00,0.750000,0,1
6,2,2014-03-01 10:00:00,0.000000,0,1
7,6,2014-03-01 11:00:00,1.000000,0,1
8,6,2014-03-01 12:00:00,0.666667,0,1
9,5,2014-03-01 13:00:00,1.000000,0,1


In [8]:
# Join with weather data
weather_file = './data/temperature/weather.csv'
weather = pd.read_csv(weather_file)
weather.drop('Unnamed: 0', axis=1, inplace=True)


In [9]:
# final_df = pd.merge(departures, weather, on='date', how='left')
final_df = pd.merge(holiday_df, weather, left_on='startdatehour', right_on = 'date', how='left')
final_df.head()

,bikeid,startdatehour,male_to_female_ratio,day_type_weekday,day_type_holiday,date,temp,fog,rain,snow,hail,thunder,tornado,visi,dewptm,humidity,wind_speed
0,1,2014-03-01 00:00:00,0,0,1,2014-03-01 00:00:00,-6.1,0,0,0,0,0,0,10,-18.9,36,5.6
1,1,2014-03-01 02:00:00,1,0,1,2014-03-01 02:00:00,-5.6,0,0,0,0,0,0,10,-15.6,46,9.3
2,1,2014-03-01 04:00:00,1,0,1,2014-03-01 04:00:00,-6.7,0,0,0,0,0,0,10,-15.6,50,0.0
3,3,2014-03-01 07:00:00,1,0,1,2014-03-01 07:00:00,-4.4,0,0,0,0,0,0,10,-13.3,51,7.4
4,1,2014-03-01 08:00:00,1,0,1,2014-03-01 08:00:00,-2.8,0,0,0,0,0,0,10,-12.2,49,7.4


In [10]:
final_df.describe()
# plt.bar(final_df.date, final_df.bikeid)
# final_df.isna()

,bikeid,male_to_female_ratio,day_type_weekday,day_type_holiday,temp,fog,rain,snow,hail,thunder,tornado,visi,dewptm,humidity,wind_speed
count,14515.000000,14515.000000,14515.000000,14515.000000,14515.000000,14515.000000,14515.000000,14515.000000,14515,14515,14515,14515.000000,14515.000000,14515.000000,14515.000000
mean,13.428384,0.849626,0.717120,0.282880,14.013669,0.001860,0.078333,0.021908,0,0,0,-231.216355,5.608743,59.958870,7.710300
std,23.720111,0.192743,0.450414,0.450414,9.426811,0.043091,0.268704,0.146389,0,0,0,1530.954438,10.425256,19.138005,5.863377
min,1.000000,0.000000,0.000000,0.000000,-17.800000,0.000000,0.000000,0.000000,0,0,0,-9999.000000,-27.800000,12.000000,0.000000
25%,3.000000,0.750000,0.000000,0.000000,6.700000,0.000000,0.000000,0.000000,0,0,0,9.000000,-1.700000,45.000000,5.600000
50%,6.000000,0.900000,1.000000,0.000000,15.000000,0.000000,0.000000,0.000000,0,0,0,10.000000,7.200000,59.000000,7.400000
75%,12.000000,1.000000,1.000000,1.000000,21.700000,0.000000,0.000000,0.000000,0,0,0,10.000000,13.900000,76.000000,11.100000
max,177.000000,1.000000,1.000000,1.000000,35.600000,1.000000,1.000000,1.000000,0,0,0,10.000000,23.900000,100.000000,37.000000


In [11]:
# # LabelEncoder for the dtstartdatehour column
# le = preprocessing.LabelEncoder()
# le.fit(final_df.dtstartdatehour)
# final_df['startdatehour'] = le.transform(final_df.dtstartdatehour)
# # LabelEncoder ends

cols = [col for col in final_df.columns if col not in ['bikeid', 'startdatehour', 'date']]

x = final_df[cols]
# y = final_df.bikeid
y = final_df.bikeid.apply(lambda x: math.log(x+1))

X_train, X_test, y_train, y_test = cross_validation.train_test_split(x, y, random_state = 1)
print cols

['male_to_female_ratio', 'day_type_weekday', 'day_type_holiday', 'temp', 'fog', 'rain', 'snow', 'hail', 'thunder', 'tornado', 'visi', 'dewptm', 'humidity', 'wind_speed']


In [12]:
###
# Linear regression
###
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
pred = regr.predict(X_test)
rmse = metrics.mean_squared_error(y_test, pred)
r2_score = metrics.r2_score(y_test, pred)

print 'MSE: ', rmse
print 'R2 score: ', r2_score

MSE:  0.895944666049
R2 score:  0.119143177265


In [13]:
###
# Ridge regression
###
ridge = linear_model.RidgeCV(alphas=[0.0001, 0.001, 0.1, 1.0, 10.0, 100.0, 1000.0], 
                             normalize=True, scoring='mean_squared_error')
ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)
print 'MSE (RidgeCV): ', metrics.mean_squared_error(y_test, ridge_pred)
ridge_r2_score = metrics.r2_score(y_test, ridge_pred)
print 'R2 Score (RidgeCV): ', ridge_r2_score

MSE (RidgeCV):  0.895903946309
R2 Score (RidgeCV):  0.119183211279


In [14]:
###
# Lasso regression
###
lasso = linear_model.Lasso(alpha = 0.00009, normalize=True)
lasso.fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)
print 'MSE (Lasso): ', metrics.mean_squared_error(y_test, lasso_pred)
lasso_r2_score = metrics.r2_score(y_test, lasso_pred)
print 'R2 Score (Lasso): ', lasso_r2_score
for i, item in enumerate(lasso.coef_):
    print cols[i], item

MSE (Lasso):  0.895754316869
R2 Score (Lasso):  0.119330320936
male_to_female_ratio 0.068482239738
day_type_weekday 0.579568698216
day_type_holiday -0.0
temp 0.0244483324896
fog 0.0
rain -0.0437604131499
snow -0.0594662185778
hail 0.0
thunder 0.0
tornado 0.0
visi 0.0
dewptm 0.0
humidity -0.000748618788123
wind_speed -0.0


In [15]:
# Based on the results above, we find that some of the coefficients are 0 and some are negative.
# We'll keep only the positive ones

rf_cols = [col for col in final_df.columns if col not in ['bikeid', 'startdatehour', 'date', 'visi', 'snow',
                                                          'hail', 'thunder', 'tornado', 'humidity', 'dewptm', 'wind_speed']]

rf_x = final_df[rf_cols]
rf_X_train, rf_X_test, rf_y_train, rf_y_test = cross_validation.train_test_split(rf_x, y, random_state = 1)

In [16]:
###
# Random Forest GridSearchCV to optimize the parameters of a classifier 
###

# Takes a lot of time and not too useful

# param_grid = { 
#     'n_estimators': [100, 200],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth': (np.array(range(8, 12))*5).tolist()
# }

# CV_rfc = grid_search.GridSearchCV(estimator=rfr, param_grid=param_grid, cv= 5)
# CV_rfc.fit(rf_X_train, rf_y_train)
# print CV_rfc.best_params_

In [17]:
###
# Random Forest regression
###
rfr = ensemble.RandomForestRegressor(n_estimators=100, bootstrap=True, 
                                     oob_score = True, random_state = 5)
rfr.fit(rf_X_train, rf_y_train)
rf_pred = rfr.predict(rf_X_test)
print 'MSE (Random Forest): ', metrics.mean_squared_error(rf_y_test, rf_pred)
rf_r2_score = metrics.r2_score(rf_y_test, rf_pred)
print 'R2 Score (Random Forest): ', rf_r2_score

# print [estimator.tree_.max_depth for estimator in rfr.estimators_]

MSE (Random Forest):  0.290005760238
R2 Score (Random Forest):  0.714877980506
